<a href="https://colab.research.google.com/github/fatemafaria142/Large-Language-Models-and-Vector-Databases-for-Retrieval-Augmented-Generation/blob/main/Text_Documents_QA_Semantic_Search_using_LangChain_and_ChromaDB_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
!pip install chromadb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 5

In [14]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.9 MB/s eta 0:00:00


### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/openchat/openchat-3.5-1210

In [4]:
mode_id = "openchat/openchat-3.5-1210"

In [5]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [6]:
tokenizer = AutoTokenizer.from_pretrained("openchat/openchat-3.5-1210")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Access Dataset**

In [7]:
!wget https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip -O stock_market_june_2023.zip
!unzip stock_market_june_2023.zip

--2024-02-07 03:49:46--  https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip [following]
--2024-02-07 03:49:46--  https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10638 (10K) [application/zip]
Saving to: ‘stock_market_june_2023.zip’

stock_market_june_2 100%[===================>]  10.39K  --.-KB/s    in 0s      

2024-02-07 03:49:46 (105 MB/s) - ‘stock_market_june_2023.zip’ saved [10638/10638]

### **Import Necessary Libraries**

In [8]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

In [9]:
# Initializes a DirectoryLoader to load text files from the specified directory.
loader = DirectoryLoader('./stock_market_june_2023/', glob="./*.txt", loader_cls=TextLoader)

# Loads all the documents present in the directory.
documents = loader.load()

#### **Chunk Size:**

* **Definition:** Chunk size is the length of each segment or unit into which a text is divided.
* **Usage:** When processing long texts or documents, breaking them into smaller chunks can be more manageable for certain algorithms, models, or systems.
* **Example:** If you set chunk_size to 100, the text will be split into consecutive segments, each containing 100 characters.

#### **Chunk Overlap:**

* **Definition:** Chunk overlap is the number of characters shared between adjacent chunks.
* **Usage:** Overlapping chunks help capture context that may span across neighboring segments, ensuring that information is not lost at the boundaries of the chunks.
* **Example:** If you set chunk_overlap to 20 and chunk_size to 100, the first 80 characters of each chunk overlap with the last 80 characters of the preceding chunk.

In [10]:
# Initialize a RecursiveCharacterTextSplitter with a specified chunk size and overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Split the loaded documents into chunks of text using the defined text_splitter.
texts = text_splitter.split_documents(documents)

# Check the length of the resulting text chunks.
print("Number of text chunks:", len(texts))


Number of text chunks: 49


### **Some Examples**

In [11]:
for i in range(3):  # Adjust the range based on the number of chunks you want to print
    print(f"{i + 1}st chunk: {texts[i].page_content}")


1st chunk: The stock market had a strong month in May, especially for those who read our previous monthly stock recommendation. At The Markets Watch, our journalists and analysts have compiled a list of monthly stock opportunities for buy-and-hold investors and traders looking to buy great companies at an even better price.
Looking at some of our stock suggestions from the previous month, the evidence of our team’s expertise becomes compelling. In May:
2st chunk: Palantir soared by over 74%
Nvidia witnessed a growth of over 34%
Crowdstrike Holdings increased by over 30%
Shopify saw a rise of over 19%
Our insights and articles about these stocks collectively yielded a remarkable 57% return last month for our readers who took action on our recommendations.
3st chunk: Institutional ETFs assembled by Goldman Sachs and Vanguard rely on growth stocks to drive their price in all market conditions, and your portfolio is no different. Most of our recommendations are already a staple in these ET

### **Let's example how well the model does at this task currently  (without training):**
* `do_sample:`This parameter controls whether to use sampling during text generation. When do_sample is set to True, the model samples words for the next token based on their probabilities. If set to False, the model uses greedy decoding and selects the token with the highest probability at each step.
* `pad_token_id:` This parameter represents the token ID used for padding in the tokenizer. In text generation, it's common to set the padding token to the end-of-sequence token (eos_token_id) to ensure that the generated text is complete.
* `top_k:` top_k controls the number of highest probability options considered during sampling. For each token, the model will limit the sampling to the top k tokens with the highest probabilities.
* **High top_k Value:** More deterministic and focused generation. The
model considers a larger number of top probability options, resulting in conservative and predictable outputs.
* **Low top_k Value:** More randomness in sampling. The model restricts the number of top probability options, leading to diverse and creative text generation, but it may result in less coherent or focused output.

* `top_p:` top_p is used in nucleus sampling, where the model considers the cumulative probability mass of the top tokens and stops sampling when it reaches a predefined threshold. It helps in preventing the model from being overly verbose and encourages diversity in the generated text.
* **High top_p Value:** Promotes diverse and varied text generation by considering a larger range of probable tokens. It's good for creating exploratory and varied outputs.
* **Low top_p Value:** Adopts a stricter threshold for selecting tokens, resulting in more conservative and focused text generation. This may lead to more deterministic outputs closely tied to the most probable tokens.
* `temperature:` temperature controls the level of randomness in the sampling process. A higher temperature (e.g., 1.0) increases randomness, making the generated text more diverse. Lower values (e.g., 0.8) make the sampling more deterministic, with the model focusing on higher probability tokens.

**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines


In [12]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_k = 50  # You can adjust this value based on your preference
top_p = 0.95  # You can adjust this value based on your preference
temperature = 0.8  # You can adjust this value based on your preference

pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024, do_sample=do_sample, pad_token_id=pad_token_id, top_k=top_k, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

### **Embedding Model**

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "intfloat/e5-large-v2"
embedding = HuggingFaceEmbeddings(model_name=model_name)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

### **Create the Database**

In [16]:
# Sets the directory where the embeddings will be stored.
persist_directory = 'database'

# Uses the Chroma module to convert the texts into embeddings using the specified HuggingFace embeddings.
# The resulting embeddings are stored in the persist_directory.
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [17]:
# Persists the generated embeddings database to disk.
vectordb.persist()

# Releases the memory held by the vectordb object by setting it to None.
vectordb = None

 #Re-initializes the Chroma object from the persisted directory with the specified HuggingFace embeddings.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

#### **Make a Retrieval**

In [26]:
# Initializes a retriever object from the vectordb with a specified number of search results (k=3).
# The vectordb.as_retriever() function is likely converting the Chroma object, which contains vectorized representations of text, into a retriever object.
# The retriever can be used to find the most similar vectors in the database given a query vector.
# The search_kwargs={"k": 3} argument suggests that the retriever will return the top 3 most similar vectors for each query.
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# create the chain to answer questions
# Initializes a RetrievalQA object with the specified HuggingFacePipeline (llm), retriever, and chain type.
# The return_source_documents parameter set to True means the original source documents will be included in the returned results.
qa_chain = RetrievalQA.from_chain_type(llm= llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)



 # This function processes the response from a Language Model (llm) and prints the result and sources.
def process_llm_response(llm_response):
    # Prints the 'result' field from the response, which likely contains the answer from the language model.
    print(llm_response['result'])

    print('\n\nSources:')
    # Iterates over the 'source_documents' field in the response, which contains the documents where the answer was found.
    for source in llm_response["source_documents"]:
        # Prints the 'source' metadata from each document.
        print(source.metadata['source'])




### **Example: 01**

In [27]:
template = """GPT4 Correct User: "Imagine you are a financial advisor, and a client seeks your guidance. Please provide detailed and accurate answers on "How has Shopify adapted to the changing business landscape, and what significant changes has it implemented recently, such as layoffs and selling its logistics business?", considering the current economic conditions and market trends. Your response should be comprehensive, well-informed, and tailored to the client's needs. <|end_of_turn|>
GPT4 Correct Assistant: """

llm_response = qa_chain(template)

process_llm_response(llm_response)

 

Shopify, an e-commerce platform, has shown remarkable resilience in adapting to the changing business landscape over the years. In response to recent challenges and market trends, Shopify has implemented a number of significant changes, including workforce layoffs and selling its logistics business. 

One major change was the workforce reduction, which saw a layoff rate of approximately 20%. This is a strategic move to streamline operations and reduce overhead costs. Though this may incur near-term expenses such as restructuring and impairment charges, it is expected to make the company more asset-light in the long run, enhancing its competitive edge.

Shopify also decided to sell its logistics business to Flexport. This was another significant change, reflecting a shift in the company's focus away from logistics and more towards its core business: providing market-leading software and solutions for its merchant clients.

Despite the challenges posed by the pandemic, Shopify has rem

### **Example: 02**

In [28]:
template = """GPT4 Correct User: "Imagine you are a financial advisor, and a client seeks your guidance. Please provide detailed and accurate answers on "What is Arbor Metals' annualized return over the past four years, and what factors contribute to its impressive performance?", considering the current economic conditions and market trends. Your response should be comprehensive, well-informed, and tailored to the client's needs. <|end_of_turn|>
GPT4 Correct Assistant: """

llm_response = qa_chain(template)

process_llm_response(llm_response)

 As a financial advisor, I would first gather the necessary data to provide the client with a comprehensive and well-informed response. However, as of my current programming, I don't have access to real-time data or the ability to calculate annualized returns. I suggest you consult a financial advisor or use a financial data platform like Yahoo Finance or Google Finance to gather the necessary information on Arbor Metals' annualized return over the past four years.

In general, to assess the performance of a company like Arbor Metals, we would consider factors such as:

1. Revenue growth: A company's ability to increase its revenue over time is a key indicator of its success and potential for future growth.

2. Earnings per share (EPS): This measures a company's profitability by dividing its net income by the number of shares outstanding. A higher EPS indicates that the company is generating more profit per share, which can be a positive indicator for investors.

3. Market trends: The 

### **Example: 03**

In [29]:
template = """GPT4 Correct User: "Imagine you are a financial advisor, and a client seeks your guidance. Please provide detailed and accurate answers on "What is Tesla's Master Plan Part 3, and how does it outline the company's long-term vision for sustainability and environmental impact?", considering the current economic conditions and market trends. Your response should be comprehensive, well-informed, and tailored to the client's needs. <|end_of_turn|>
GPT4 Correct Assistant: """

llm_response = qa_chain(template)

process_llm_response(llm_response)

 

Tesla's Master Plan Part 3, unveiled by Elon Musk, is a long-term vision that aims to nurture a sustainable future without inflicting environmental harm. The plan anticipates a massive $10 trillion investment in various sectors, which Musk believes is feasible and "not a big number relative to the global economy." The plan is centered around three main pillars:

1. Autonomy: Tesla plans to achieve full autonomy in its vehicles, which will revolutionize the transportation industry and potentially lead to a peak margin of 80% in its automotive segment.

2. Energy: Tesla Energy aims to generate a robust 20-30% margin, focusing on providing clean energy solutions to homes, businesses, and industries.

3. Affordable Lithium Supply: Tesla is also looking to tap into affordable lithium suppliers like Arbor Metals to drive unmatched profit growth and future-proofing in the upcoming decades as the lithium and electric vehicle (EV) industries continue to grow exponentially.

Considering the c